In [ ]:
from typing import List
from numpy import mean
from sentence_transformers import SentenceTransformer, util

embed_model = SentenceTransformer('sentence-transformers/gtr-t5-large')

In [ ]:
def is_in_topic(
    new_msg: str,
    ctx_list: List[str],
    decay_rate: float = 0.85,
    max_threshold: float = 0.85,
    mean_threshold: float = 0.75,
    verbose: bool = True,
):
    if not ctx_list: return True

    ctx_emb = embed_model.encode(ctx_list, convert_to_tensor=True)
    msg_emb = embed_model.encode(new_msg, convert_to_tensor=True)

    similarities = util.cos_sim(msg_emb, ctx_emb)[0]

    weights = [decay_rate**i for i in reversed(range(len(ctx_list)))]
    weighted_similarities = [sim.item() * w for sim, w in zip(similarities, weights)]

    max_sim = max(weighted_similarities)
    mean_sim = mean(weighted_similarities)

    if verbose:
        print(f"Current message: {new_msg}")
        print(f"Weighted Similarities")

        for i, (msg, score) in enumerate(zip(ctx_list, weighted_similarities)):
            print(f" [{i}] {msg[:40]}... -> {score:.4f}")

        print(f"\n Max Sim: {max_sim:.4f}, Mean Sim: {mean_sim:.4f}")
        print(
            f" Decision: {'In Context' if (max_sim > max_threshold and mean_sim > mean_threshold) else 'New Topic'}\n"
        )

    return max_sim > max_threshold and mean_sim > mean_threshold

# Simulate message input
messages = [
    # "我上星期去了台北玩。",
    # "天氣很熱，但是很好玩。",
    # "我還去了故宮，裡面有很多古董。",
    
    # "下週學校要開始期中考了。",
    # "我數學還沒讀完，有點緊張。",
    # "最近睡不好，常常熬夜。",
    # "醫生建議我早點睡，多運動。",
    
    # "我昨天遇到小美。",
    # "她說她也去了台北出差。",
    
    "今天天氣怎麼樣？",
    "今天天氣很好，陽光明媚。",
    "是啊，這樣的天氣真讓人心情愉快。",
    "對，適合外出走走。",
    
    "你最近有去哪裡旅行嗎？",
    "有，我去了京都。",
    "哇，那裡怎麼樣？",
    "非常美麗，特別是秋天的紅葉。",
    
    "你最近身體怎麼樣？",
    "還不錯，就是有點累。",
    "那你有去運動嗎？",
    "有，我每天早上跑步。",
    
    "你最近看過什麼好電影嗎？",
    "有，我看了《流浪地球》。",
    "怎麼樣？好看嗎？",
    "非常精彩，特效做得很棒。",
]

conversation = []

for message in messages:
    res = is_in_topic(message, conversation)
    
    if (res):
        conversation.append(message)
    else:
        conversation = [message]

In [ ]:
0.85 ** 5